In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 35.3 MB/s 
     |████████████████████████████████| 596 kB 64.4 MB/s 
     |████████████████████████████████| 6.6 MB 55.6 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install hazm

In [ ]:
import pandas as pd
import csv
import re
import json
import os
import numpy as np
import hazm

import collections
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Data preparation

In [ ]:
with open(f'/content/drive/MyDrive/dataset_annotated_sentiment.json', encoding='utf-8') as fh:
        data = json.load(fh)

In [ ]:
with open(f'/content/drive/MyDrive/back_trans_fa_en_fa.json', encoding='utf-8') as fh:
        upsampled_data = json.load(fh)

In [ ]:
persian_to_english = {
    '۰': '0',
    '۱': '1',
    '۲': '2',
    '۳': '3',
    '۴': '4',
    '٤': '4',
    '۵': '5',
    '٥': '5',
    '۶': '6',
    '٦': '6',
    '۷': '7',
    '۸': '8',
    '۹': '9'
}
def preprocess(text):
    #stripping
    # text = text.strip()
    #removing newlines
    # text = text.replace("\n"," ")
    #removing hashtags
    text = re.sub("#", "", text)
    #removing extra spaces
    # text = re.sub("\s+", " ", text)
    #removing url
    url_re = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text = re.sub(url_re, '', text)
    #removing html tags
    html_re = r"(<.*?>)"
    text = re.sub(html_re, '', text)
    #removing _ character
    text = text.replace("_","")
    #removing " character
    text = text.replace("\"","")
    #removing « character
    text = text.replace("«","")
    #removing »  character
    text = text.replace("»","")    
    #removing latin characters and tab
    text = re.sub("([A-Za-z\t])", "", text)
    #removing repeating ـ which is often used for keshidan like تـــابع
    text = re.sub('ـ*', '', text)
    #removing references like [۱]
    text = re.sub("\[[۱۲۳۴۵۶۷۸۹۰1234567890]*\]"," ", text)
    #converting persian and arabic numbers to english
    for p in persian_to_english.keys():
        text = text.replace(p,persian_to_english[p])
    #normalization
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)

    return text




In [ ]:
def get_eval_final_tag():
    not_tagged = 0
    eval_data = []
    for news in data["eval"]:
        s = 0
        if news["annotations"] == []:
            not_tagged +=1
            continue   
        news["text"] = preprocess(news["text"])
        for annotation in news["annotations"]:
            if annotation == 'خنثی':
              s+=0
            elif annotation == 'منفی':
              s+=-1
            else:
              s+=1
        if s < 0:
          tag = 'منفی'
        elif s ==0 :
          tag = 'خنثی'
        else:
          tag = 'مثبت'

        eval_data.append((news["text"],news["annotations"],tag))
        
    return eval_data, not_tagged

In [ ]:
eval_data, not_tagged = get_eval_final_tag()

In [ ]:
def get_test_final_tag():
    not_tagged = 0
    test_data = []
    for news in data["test"]:
        s = 0
        if news["annotations"] == []:
            not_tagged +=1
            continue   
        news["text"] = preprocess(news["text"])
        for annotation in news["annotations"]:
            if annotation == 'خنثی':
              s+=0
            elif annotation == 'منفی':
              s+=-1
            else:
              s+=1
        if s < 0:
          tag = 'منفی'
        elif s ==0 :
          tag = 'خنثی'
        else:
          tag = 'مثبت'

        test_data.append((news["text"],news["annotations"],tag))
        
    return test_data, not_tagged

In [ ]:
test_data, not_tagged = get_test_final_tag()

In [ ]:
upsampled_train_data = []
for news in upsampled_data["train"]:
    news["text"] = preprocess(news["text"])
    upsampled_train_data.append((news["text"], news["annotations"], news["fa"]))

In [ ]:
train_header = ['text', 'annotations', 'final_tag']  
with open('upsampled_train_data.csv', 'w', newline='', encoding='utf-8') as f: 
    write = csv.writer(f) 
    write.writerow(train_header) 
    write.writerows(upsampled_train_data) 

In [ ]:
eval_header = ['text', 'annotations', 'final_tag']  

with open('eval_data.csv', 'w', newline='', encoding='utf-8') as f: 
    write = csv.writer(f) 
    write.writerow(eval_header) 
    write.writerows(eval_data) 

In [ ]:
test_header = ['text', 'annotations', 'final_tag']  

with open('test_data.csv', 'w', newline='', encoding='utf-8') as f: 
    write = csv.writer(f) 
    write.writerow(test_header) 
    write.writerows(test_data) 

In [ ]:
upsampled_df = pd.read_csv('upsampled_train_data.csv')
upsampled_df.head()

,text,annotations,final_tag
0,آتش سوزی کارخانه جمیل نخ گسترده است / اعزام نی...,"['منفی', 'منفی', 'منفی']",منفی
1,تجهیزات باکیفیت صنعتی آشپزخانه و کافی شاپ\n\nب...,"['مثبت', 'مثبت', 'خنثی']",مثبت
2,برگزاری جشنواره رسانه ابوذر در دی ماه سال جاری...,"['مثبت', 'خنثی', 'خنثی']",مثبت
3,افزایش ۸۰ درصدی فروش سلاح در آمریکا در ژانویه ...,"['منفی', 'منفی', 'منفی']",منفی
4,بازداشت دو تروریست که در بمب گذاری مسیر زائران...,"['خنثی', 'مثبت']",مثبت


In [ ]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import hazm
upsampled_df['news_len_by_words'] = upsampled_df['text'].apply(lambda t: len(hazm.word_tokenize(t)))

In [ ]:
min_max_len = upsampled_df["news_len_by_words"].min(), upsampled_df["news_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 12 	Max: 592


In [ ]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='news_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [ ]:
data_gl_than(upsampled_df, 256, 3)

Texts with word length of greater than 3 and less than 256 includes 98.03% of the whole!


In [ ]:
eval_df = pd.read_csv('eval_data.csv')
eval_df.head()

,text,annotations,final_tag
0,نجات از عوارض غذا‌های فرآوری شده با امگا ۳؟ \n...,"['خنثی', 'منفی', 'خنثی']",منفی
1,معاون رئیس جمهور: دولت از ظرفیت نهادهای انقلاب...,"['مثبت', 'مثبت', 'مثبت']",مثبت
2,شرکت‌های دانش بنیان ضربات اقتصادی همه گیری کرو...,"['مثبت', 'مثبت', 'مثبت']",مثبت
3,الهام علی اف: ایران کشور دوست و برادر جمهوری ...,"['مثبت', 'مثبت', 'مثبت']",مثبت
4,مرگ یک کارگر چاه کن بر اثر ریزش چاه\n\nبه گزار...,"['منفی', 'منفی', 'منفی']",منفی


In [ ]:
test_df = pd.read_csv('test_data.csv')
test_df.head()

,text,annotations,final_tag
0,رکوردشکنی رمزارز دوم\n\nجریان مداوم اخبار دربا...,"['مثبت', 'خنثی', 'مثبت']",مثبت
1,نشست گستره قلمرو و حکم حاکم در مذهب امامیه و ح...,"['خنثی', 'مثبت', 'خنثی']",مثبت
2,اعلام آمادگی دفتر آیت الله سیستانی برای کمک به...,"['خنثی', 'خنثی']",خنثی
3,چشم جهان نمای ایران با منظومه سازی ماهواره‌ها ...,"['مثبت', 'مثبت']",مثبت
4,اجرای ۲ پروژه از طرح‌های پتروپالایش در خوزستان...,"['مثبت', 'مثبت', 'مثبت']",مثبت


In [ ]:
upsampled_df_positive = upsampled_df[upsampled_df['final_tag']=='مثبت']
upsampled_df_neutral = upsampled_df[upsampled_df['final_tag']=='خنثی']
upsampled_df_negative = upsampled_df[upsampled_df['final_tag']=='منفی']


In [ ]:
print(len(upsampled_df_positive),len(upsampled_df_neutral),len(upsampled_df_negative))


608 608 608


In [ ]:
upsampled_train_df = pd.concat([upsampled_df_negative, upsampled_df_positive, upsampled_df_neutral])
upsampled_train_df = upsampled_df.sample(frac=1).reset_index(drop=True)
upsampled_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1824 entries, 0 to 1823
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               1824 non-null   object
 1   annotations        1824 non-null   object
 2   final_tag          1824 non-null   object
 3   news_len_by_words  1824 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 57.1+ KB


In [ ]:
upsampled_train_df.head()
upsampled_train_df.to_csv('upsampled_balanced_data.csv', encoding='utf-8')


In [ ]:
x_train, y_train = upsampled_train_df['text'].values.tolist(), upsampled_train_df['final_tag'].values.tolist()
x_valid, y_valid = eval_df['text'].values.tolist(), eval_df['final_tag'].values.tolist()
x_test, y_test = test_df['text'].values.tolist(), test_df['final_tag'].values.tolist()

In [ ]:
print(upsampled_df.shape)
print(eval_df.shape)
print(test_df.shape)

(1824, 4)
(75, 3)
(75, 3)


In [ ]:
####################################### Transformer ########################################

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [ ]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 7
EEVERY_EPOCH = 1000
LEARNING_RATE = 1e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-hw4/pytorch_model.bin'
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [ ]:
label2id = {'منفی': 0, 'مثبت': 1, 'خنثی':2}
id2label = {0: 'منفی', 1: 'مثبت', 2: 'خنثی'}



In [ ]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\u0645\u0646\u0641\u06cc",
    "1": "\u0645\u062b\u0628\u062a",
    "2": "\u062e\u0646\u062b\u06cc"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "\u062e\u0646\u062b\u06cc": 2,
    "\u0645\u062b\u0628\u062a": 1,
    "\u0645\u0646\u0641\u06cc": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



In [ ]:
idx = np.random.randint(0, len(upsampled_train_df))
sample_text = upsampled_train_df.iloc[idx]['text']
sample_tag = upsampled_train_df.iloc[idx]['final_tag']

print(f'Sample: \n{sample_text}\n{sample_tag}')

Sample: 
طرح‌های مجلس برای حمایت معیشتی از مردم/ پرداخت یارانه کالایی به مردم

به گزارش خبرگزاری فارس از آذرشهر، حجت الاسلام والمسلمین سیدمحمدرضا میرتاج الدینی با اشاره به اینکه حل مشکلات اقتصادی جامعه جزو وظایف اصلی حکومت‌ها است، اظهار داشت: درخصوص عملکرد ۶ ماهه مجلس شورای اسلامی، به همراه همکاران محترم ۱۰ اولویت را انتخاب کرده و پیش میبریم که اولین اولویت ما حمایت از معیشت مردم است که در چند ساله اخیر به دلیل تکانه‌های اقتصادی سرمایه مردم کاهش یافته است. وی با اشاره به اینکه آیت الله مصباح یزدی در پیش و پس از پیروزی انقلاب و عصر شبهه‌های عقیدتی و ایدئولوژیک علیه دین و انقلاب با ارتباط با جوانان در تثبیت اندیشه‌های دینی اهتمام فراوانی داشت، افزود: ایشان در دفاع از اسلام به صورت علمی و فلسفی و تبعیت از امام و رهبری و در مسیر انقلاب ثابت قدم بود و ما باید با الگوپذیری از علمایی چون ایشان و سرداران مقاومت و شخصیت عزیزی چون سردار حاج قاسم سلیمانی این مسیر را ادامه داده و تلاش کنیم هم بر مشکلات داخلی غلبه کنیم و هم عزت اسلام و انقلاب و نظام اسلامی را در دنیا حفظ کنیم
مثبت


In [ ]:
tokens = tokenizer.tokenize(sample_text)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  Text: {sample_text}\n')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}\n')
print(f'Token IDs: {token_ids}')

  Text: طرح‌های مجلس برای حمایت معیشتی از مردم/ پرداخت یارانه کالایی به مردم

به گزارش خبرگزاری فارس از آذرشهر، حجت الاسلام والمسلمین سیدمحمدرضا میرتاج الدینی با اشاره به اینکه حل مشکلات اقتصادی جامعه جزو وظایف اصلی حکومت‌ها است، اظهار داشت: درخصوص عملکرد ۶ ماهه مجلس شورای اسلامی، به همراه همکاران محترم ۱۰ اولویت را انتخاب کرده و پیش میبریم که اولین اولویت ما حمایت از معیشت مردم است که در چند ساله اخیر به دلیل تکانه‌های اقتصادی سرمایه مردم کاهش یافته است. وی با اشاره به اینکه آیت الله مصباح یزدی در پیش و پس از پیروزی انقلاب و عصر شبهه‌های عقیدتی و ایدئولوژیک علیه دین و انقلاب با ارتباط با جوانان در تثبیت اندیشه‌های دینی اهتمام فراوانی داشت، افزود: ایشان در دفاع از اسلام به صورت علمی و فلسفی و تبعیت از امام و رهبری و در مسیر انقلاب ثابت قدم بود و ما باید با الگوپذیری از علمایی چون ایشان و سرداران مقاومت و شخصیت عزیزی چون سردار حاج قاسم سلیمانی این مسیر را ادامه داده و تلاش کنیم هم بر مشکلات داخلی غلبه کنیم و هم عزت اسلام و انقلاب و نظام اسلامی را در دنیا حفظ کنیم

   Tokens: طرحهای مجلس

In [ ]:
encoding = tokenizer.encode_plus(
    sample_text,
    max_length=32,
    truncation=True,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt',  # Return PyTorch tensors
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[    2,  6395,  3590,  2831,  3876, 18249,  2791,  3155,  1013,  3486,
          7681, 10911,  2789,  3155,  2789,  3361,  6000,  4922,  2791, 43242,
          1348,  7923,  9877, 18240, 71962, 92606, 51433,  2799,  3364,  2789,
          3131,     4]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])


In [ ]:
class newsDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for news. """

    def __init__(self, tokenizer, texts, targets=None, label_list=None, max_len=128):
        self.texts = texts
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len

        
        self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])

        if self.has_target:
            target = self.label_map.get(str(self.targets[item]), str(self.targets[item]))

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
        
        inputs = {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)
        
        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = newsDataset(
        texts=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len, 
        label_list=label_list)
    
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
label_list = ['منفی', 'مثبت','خنثی']
train_data_loader = create_data_loader(upsampled_train_df['text'].to_numpy(), upsampled_train_df['final_tag'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(eval_df['text'].to_numpy(), eval_df['final_tag'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test_df['text'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

In [ ]:
class SentimentModel(nn.Module):

    def __init__(self, config):
        super(SentimentModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH,return_dict=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids)
        
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits 

In [ ]:
pt_model = SentimentModel(config=config)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pt_model <class '__main__.SentimentModel'>


In [ ]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)

    return y, loss


def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):
            
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            # calculate the batch loss
            loss = loss_fn(outputs, targets)

            # accumulate all the losses
            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)
    
    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss


def train_op(model, 
             data_loader, 
             loss_fn, 
             optimizer, 
             scheduler, 
             step=0, 
             print_every_step=100, 
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None, 
             clip=0.0):
    
    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        # move tensors to GPU if CUDA is available
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        # clear the gradients of all optimized variables
        optimizer.zero_grad()

        # compute predicted outputs by passing inputs to the model
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
        # convert output probabilities to predicted class
        _, preds = torch.max(outputs, dim=1)

        # calculate the batch loss
        loss = loss_fn(outputs, targets)

        # accumulate all the losses
        losses.append(loss.item())

        # compute gradient of the loss with respect to model parameters
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        # perform optimization step
        optimizer.step()

        # perform scheduler step
        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

In [ ]:
optimizer = AdamW(pt_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)
        
        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))
            
            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss
        
        return eval_loss_min


    return eval_cb


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model, 
        data_loader=train_data_loader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader, 
        clip=CLIP)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=pt_model, 
        data_loader=valid_data_loader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Training... : 100%|██████████| 114/114 [00:48<00:00,  2.36it/s]

Training... : 100%|██████████| 114/114 [00:48<00:00,  2.34it/s]

Training... : 100%|██████████| 114/114 [00:50<00:00,  2.27it/s]

Training... : 100%|██████████| 114/114 [00:48<00:00,  2.36it/s]

Training... : 100%|██████████| 114/114 [00:48<00:00,  2.36it/s]

Training... : 100%|██████████| 114/114 [00:49<00:00,  2.30it/s]

Training... : 100%|██████████| 114/114 [00:48<00:00,  2.36it/s]

Epochs... : 100%|██████████| 7/7 [05:47<00:00, 49.67s/it]


In [ ]:
def predict(model, comments, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size, None)
    
    predictions = []
    prediction_probs = []

    
    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            
            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [ ]:
test_news = test_df['text'].to_numpy()
preds, probs = predict(pt_model, test_news, tokenizer, max_len=128)

print(preds.shape, probs.shape)

100%|██████████| 3/3 [00:00<00:00,  5.53it/s]

(75,) (75, 3)


In [ ]:
y_test, y_pred = [label_list.index(label) for label in test_df['final_tag'].values], preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print()
print(classification_report(y_test, y_pred, target_names=label_list))

F1: 0.706233740589905

              precision    recall  f1-score   support

        منفی       0.67      0.74      0.70        19
        مثبت       0.81      0.78      0.79        37
        خنثی       0.56      0.53      0.54        19

    accuracy                           0.71        75
   macro avg       0.68      0.68      0.68        75
weighted avg       0.71      0.71      0.71        75



In [ ]:
label_list

['منفی', 'مثبت', 'خنثی']

In [ ]:
confusion_matrix(y_test, y_pred)

array([[14,  2,  3],
       [ 3, 29,  5],
       [ 4,  5, 10]])